In [1]:
import pandas as pd
reviews =  "reviews.csv"

df = pd.read_csv(reviews)
df.head()
data = df['Text']
df.head()

,Sentiment,Time,Text
0,positive,18/6/21,This is a very healthy dog food. Good for thei...
1,positive,7/7/21,I've been very pleased with the Natural Balanc...
2,positive,18/6/21,"Before I was educated about feline nutrition, ..."
3,positive,7/7/21,"My holistic vet recommended this, along with a..."
4,positive,1/7/21,I bought this coffee because its much cheaper ...


In [ ]:
#!pip install umap-learn

In [2]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('all-MiniLM-L6-v2')
model

SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Normalize()
)

In [4]:
from sentence_transformers import SentenceTransformer
from umap import UMAP
from hdbscan import HDBSCAN

embedding_model = SentenceTransformer('all-MiniLM-L6-v2')
umap_model = UMAP(n_neighbors=3, n_components=3, min_dist=0.05)
hdbscan_model = HDBSCAN(min_cluster_size=80, min_samples=40,
                        gen_min_span_tree=True,
                        prediction_data=True)

In [7]:
import nltk
nltk.download('stopwords')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

You might notice that we have added prediction_data=True as a new parameter to HDBSCAN. We need this to avoid an AttributeError when integrating our custom HDBSCAN step with BERTopic. Adding gen_min_span_tree adds another step to HDBSCAN that can improve the resultant clusters.
We must also initialize a vectorizer_model to handle stopword removal during the c-TF-IDF step. We will use the list of stopwords from NLTK but add a few more tokens that seem to pollute the results.

In [8]:
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords

stopwords = list(stopwords.words('english')) + ['http', 'https', 'amp', 'com']

# we add this to remove stopwords that can pollute topcs
vectorizer_model = CountVectorizer(ngram_range=(1, 2), stop_words=stopwords)

In [16]:
from bertopic import BERTopic

model = BERTopic(
    umap_model=umap_model,
    hdbscan_model=hdbscan_model,
    embedding_model=embedding_model,
    vectorizer_model=vectorizer_model,
    top_n_words=10,
    language='english',
    calculate_probabilities=True,
    verbose=True
)
topics, probs = model.fit_transform(data)

Batches:   0%|          | 0/171 [00:00<?, ?it/s]

2023-04-11 10:06:37,038 - BERTopic - Transformed documents to Embeddings
2023-04-11 10:06:42,553 - BERTopic - Reduced dimensionality
2023-04-11 10:06:43,067 - BERTopic - Clustered reduced embeddings


In [21]:
model.visualize_barchart()

In [22]:
model.get_topics()

{-1: [('br', 0.020432171743205742),
  ('like', 0.012911178151771648),
  ('good', 0.01192014073796087),
  ('br br', 0.01150960825727352),
  ('taste', 0.011502025673670159),
  ('great', 0.011431796105177734),
  ('product', 0.011181248222938743),
  ('one', 0.01030516548037011),
  ('water', 0.008972615029747425),
  ('amazon', 0.008452213248903986)],
 0: [('coffee', 0.07099736059063634),
  ('cup', 0.026031579242692504),
  ('cups', 0.018501693212124665),
  ('like', 0.017150919243419438),
  ('br', 0.016778939178536446),
  ('taste', 0.01655885790550883),
  ('beans', 0.014495677980546214),
  ('keurig', 0.013729987661025804),
  ('flavor', 0.012910142107263616),
  ('good', 0.01288990218701376)],
 1: [('tea', 0.09713828034569144),
  ('br', 0.020928645957780056),
  ('like', 0.018749203749325354),
  ('taste', 0.017377639173797282),
  ('green', 0.016853753738157375),
  ('teas', 0.016542490120736632),
  ('green tea', 0.015758844306376394),
  ('flavor', 0.014551405108326837),
  ('br br', 0.013656741895

In [23]:
HDBSCAN_UMAP_BERT_TOPIC = pd.DataFrame.from_dict(model.get_topics())

In [24]:
HDBSCAN_UMAP_BERT_TOPIC.to_csv('top_10_HDBSCAN_UMAP_BERT_TOPIC.csv', index=False)